In [40]:
import napari
print("Napari version no.:", napari.__version__)
import os, glob, enum
import numpy as np
import tifffile as tiff
from tqdm import tqdm
from skimage.io import imread
from natsort import natsorted
from datetime import datetime

Napari version no.: 0.4.7


In [21]:
data_folder = '/home/nathan/data/kraken/pcna/21_04_29/Pos2'#'/media/nathan/Transcend/Nathan/data/pcna/test_210422/Pos7'
output_dir = os.path.join(data_folder,'stacks')

In [16]:
@enum.unique
class channels(enum.Enum):
#     bf = 0 
#     gfp = 1
#     rfp = 2
#     irfp = 3
    irfp = 0

In [23]:
## compile stacks
if 'stacks' not in os.listdir(data_folder):
    os.mkdir(os.path.join(data_folder,'stacks'))
    output_dir = os.path.join(data_folder,'stacks')
    print(output_dir, 'directory created')
    for channel in channels:
        print(channel.name, 'compiling')
        files = glob.glob(os.path.join(data_folder,'img_channel00{}*.tif'.format(channel.value)))
        stack = []
        for file in tqdm(natsorted(files)):
            tif = np.array(tiff.imread(file),dtype=np.uint8)
            tif_w,tif_h = np.shape(tif)[0],np.shape(tif)[1]
            tif = tif[int((tif_w-1200)/2):int(tif_w-(tif_w-1200)/2),int((tif_h-1600)/2):int(tif_h-(tif_h-1600)/2)] ## cropping
            stack.append(tif)
        stack = np.stack(stack,axis=0)
        with tiff.TiffWriter(os.path.join(output_dir,'{}.tif'.format(channel.name)),imagej=True) as output_file:
            output_file.save(stack)
        print(channel.name, 'saved out')
else:
    print('looks like stacks already exist bro')

 20%|█▉        | 71/360 [00:00<00:00, 704.89it/s]

/home/nathan/data/kraken/pcna/21_04_29/Pos2/stacks directory created
irfp compiling


100%|██████████| 360/360 [00:00<00:00, 632.23it/s]


irfp saved out


In [25]:
### load images
stack_fn = {}
for channel in channels:
    stack_fn[channel.name] = glob.glob(os.path.join(output_dir, '{}.tif'.format(channel.name)))
#gfp = imread(stack_fn['gfp'][0])
#rfp = imread(stack_fn['rfp'][0])
irfp = imread(stack_fn['irfp'][0])

In [41]:
### launch napari
with napari.gui_qt():
    viewer = napari.Viewer()

    #viewer.add_image(bf, name = "brightfield")
    #viewer.add_image(gfp, name="gfp", blending = "additive", colormap = "green")
    #viewer.add_image(rfp, name="rfp", contrast_limits = (0, 50), blending = "additive", colormap = "magenta")
    viewer.add_image(irfp, name="irfp", blending = "additive", colormap = "cyan")
    
    @viewer.bind_key('m')
    def make_movie(viewer):
        """Make a movie."""
        num_frames = int(viewer.dims.range[0][1])
        def _make_movie():
            date = datetime.now().strftime("%Y_%m_%d-%I:%M:%S_%p")
            save_folder = f'/home/nathan/Documents/presentations/figures_animations/movie_{date}'
            os.mkdir(save_folder)
            for i in range(num_frames+1):
                viewer.dims.set_point(0, i)
                fn = os.path.join(save_folder, f'movie_{i}.png')
                image = viewer.screenshot(path=fn, canvas_only=True)
            print('Movie saved out')
        _make_movie()

Movie saved out
